In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-04-10 21:05:59--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   110MB/s    in 0.2s    

2023-04-10 21:05:59 (110 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code
# we call the dataframe to visualize the dataframe and its original shape
df_books






,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber
...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato


In [5]:
df_ratings

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0
...,...,...,...
1149775,276704,1563526298,9.0
1149776,276706,0679447156,0.0
1149777,276709,0515107662,10.0
1149778,276721,0590442449,10.0


In [8]:
# cleaning the data 
df_books_and_ratings = pd.merge (df_ratings, df_books, on = 'isbn', how= 'left')
df_books_and_ratings

,user,isbn,rating,title,author
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
1,276726,0155061224,5.0,Rites of Passage,Judith Rae
2,276727,0446520802,0.0,The Notebook,Nicholas Sparks
3,276729,052165615X,3.0,Help!: Level 1,Philip Prowse
4,276729,0521795028,6.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather
...,...,...,...,...,...
1149775,276704,1563526298,9.0,Get Clark Smart : The Ultimate Guide for the S...,Clark Howard
1149776,276706,0679447156,0.0,Eight Weeks to Optimum Health: A Proven Progra...,Andrew Weil
1149777,276709,0515107662,10.0,The Sherbrooke Bride (Bride Trilogy (Paperback)),Catherine Coulter
1149778,276721,0590442449,10.0,Fourth Grade Rats,Jerry Spinelli


In [19]:
#remove books with less than 100 ratings from dfbooks
# get the total rates per book
times_rated = df_books_and_ratings['isbn'].value_counts ()
display (times_rated)
times_rated = times_rated [times_rated >100].index
times_rated


0971880107     2502
0316666343     1295
0385504209      883
0060928336      732
0312195516      723
               ... 
1568656386        1
1568656408        1
1569551553        1
1570081808        1
05162443314       1
Name: isbn, Length: 340556, dtype: int64

Index(['0971880107', '0316666343', '0385504209', '0060928336', '0312195516',
       '044023722X', '0679781587', '0142001740', '067976402X', '0671027360',
       ...
       '080411918X', '0140042598', '0446604844', '076790592X', '0385508417',
       '0446530522', '0385335830', '0312924801', '051511992X', '0553208845'],
      dtype='object', length=721)

In [22]:
#remove users with less than 200 ratings
users_high_rates = df_books_and_ratings ['user'].value_counts()
display (users_high_rates) 
users_high_rates = users_high_rates [users_high_rates >200].index
users_high_rates


11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: user, Length: 105283, dtype: int64

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
            260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727,
            268622, 188951],
           dtype='int64', length=899)

In [24]:
#keep cleaning the dataframe
df_filtered = df_books_and_ratings.loc [(df_books_and_ratings['isbn'].isin (times_rated.values)) & (df_books_and_ratings['user'].isin(users_high_rates.values))]
df_filtered



,user,isbn,rating,title,author
1456,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1469,277427,0060930535,0.0,The Poisonwood Bible: A Novel,Barbara Kingsolver
1471,277427,0060934417,0.0,Bel Canto: A Novel,Ann Patchett
1474,277427,0061009059,9.0,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich
1484,277427,0140067477,0.0,The Tao of Pooh,Benjamin Hoff
...,...,...,...,...,...
1147304,275970,0804111359,0.0,Secret History,DONNA TARTT
1147436,275970,140003065X,0.0,A Fine Balance,Rohinton Mistry
1147439,275970,1400031346,0.0,The No. 1 Ladies' Detective Agency,Alexander McCall Smith
1147440,275970,1400031354,0.0,Tears of the Giraffe (No.1 Ladies Detective Ag...,Alexander McCall Smith


In [27]:
#looking for duplicates
df_filtered = df_filtered.drop_duplicates(['title', 'user'])
df_filtered



,user,isbn,rating,title,author
1456,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1469,277427,0060930535,0.0,The Poisonwood Bible: A Novel,Barbara Kingsolver
1471,277427,0060934417,0.0,Bel Canto: A Novel,Ann Patchett
1474,277427,0061009059,9.0,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich
1484,277427,0140067477,0.0,The Tao of Pooh,Benjamin Hoff
...,...,...,...,...,...
1147304,275970,0804111359,0.0,Secret History,DONNA TARTT
1147436,275970,140003065X,0.0,A Fine Balance,Rohinton Mistry
1147439,275970,1400031346,0.0,The No. 1 Ladies' Detective Agency,Alexander McCall Smith
1147440,275970,1400031354,0.0,Tears of the Giraffe (No.1 Ladies Detective Ag...,Alexander McCall Smith


In [30]:
# remove missing values
df_filtered = df_filtered.dropna(how= 'any')
df_filtered

,user,isbn,rating,title,author
1456,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1469,277427,0060930535,0.0,The Poisonwood Bible: A Novel,Barbara Kingsolver
1471,277427,0060934417,0.0,Bel Canto: A Novel,Ann Patchett
1474,277427,0061009059,9.0,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich
1484,277427,0140067477,0.0,The Tao of Pooh,Benjamin Hoff
...,...,...,...,...,...
1147304,275970,0804111359,0.0,Secret History,DONNA TARTT
1147436,275970,140003065X,0.0,A Fine Balance,Rohinton Mistry
1147439,275970,1400031346,0.0,The No. 1 Ladies' Detective Agency,Alexander McCall Smith
1147440,275970,1400031354,0.0,Tears of the Giraffe (No.1 Ladies Detective Ag...,Alexander McCall Smith


In [32]:
books = ["Where the Heart Is (Oprah's Book Club (Paperback))",
        "I'll Be Seeing You",
        "The Weight of Water",
        "The Surgeon",
        "I Know This Much Is True"]

for book in books:
  print(df_filtered.isbn.isin(df_filtered[df_filtered.title == book].isbn).sum())

183
75
49
57
77


In [34]:
#prepare teh dataset for the model
df_wide_format = df_filtered.pivot (index= 'title', columns= 'user', values= 'rating').fillna(0)
df_wide_format

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
#Building the model
model = NearestNeighbors (metric= 'cosine')
model.fit (df_wide_format)


NearestNeighbors(metric='cosine')

In [48]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = [book,[]]

  distance, title = model.kneighbors ([df_wide_format.loc[book]], n_neighbors= 6, return_distance= True)
  books_titles = df_wide_format.iloc [np.flip(title[0])][:-1].index.tolist()
  books_distance = list(np.flip(distance[0])[:-1])

  for titles in zip (books_titles, books_distance):
    recommended_books [1].append (list(titles))

  return recommended_books

In [54]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7230184]]]
You passed the challenge! 🎉🎉🎉🎉🎉
